In [42]:
import streamlit as st
import gpxpy
import folium
from streamlit_folium import st_folium
import pandas as pd
import boto3
import json
import random



def parse_gpx(gpxdf, number):
    points = []
    gpx_data = gpx = gpxpy.parse(gpxdf['activity_gpx']) 
    color = gpxdf['color']
    filename = gpxdf['filename']
    
    for track in gpx_data.tracks:
        for segment in track.segments:
            for point in segment.points:
                points.append({'Latitude': point.latitude, 'Longitude': point.longitude, 'Color': color, 'Number' : number})
    return points

def bucket_query_namefiles():
    activity_filename = []
    session = boto3.Session()
    s3 = session.resource('s3') 
    bucket = s3.Bucket('solvesdgs')
    for obj in bucket.objects.all():
        if 'activityfiles' in obj.key:
            activity_filename.append(obj.key)
    return activity_filename[1:]

def generate_random_color():
    # Generate a random hexadecimal color code
    color_code = "#{:06x}".format(random.randint(0, 0xFFFFFF))
    return color_code

def download_files():
    session = boto3.Session()
    s3 = session.resource('s3') 
    files_name = bucket_query_namefiles()
    json_df = pd.DataFrame()
    color_random = generate_random_color()
    for obj_name in files_name:
        content_object = s3.Object('solvesdgs', obj_name)
        file_content = content_object.get()['Body'].read()    
        json_content = json.loads(file_content)
        json_content['filename'] = obj_name
        json_content['color'] = color_random
        content_df = pd.DataFrame([json_content],  index=[0])
        content_df= content_df[['activity_gpx','filename', 'color' ]]
        json_df = pd.concat([json_df,content_df], ignore_index=True)
    return json_df

def read_files():
    df_content = download_files()
    # Parse each GPX file
    all_points = []    # for file, color in colors.items():
    for index, row in df_content.iterrows():
        points = parse_gpx(row, index)
        all_points.extend(points)

    return all_points

def plot_map(points):
    m = folium.Map(location=[points[0]['Latitude'], points[0]['Longitude']], zoom_start=12)
    seen = {}
    for point in points:
        if not seen.get(point['Number'], False):
            folium.Marker(location=[point['Latitude'], point['Longitude']],
                      icon=folium.Icon(color=point['Color'], icon='info-sign')
                      ).add_to(m)
        seen[point['Number']] = True

    grouped = pd.DataFrame(points).groupby('Number')

    for number_id, group_data in grouped:
        color_1 = group_data['Color'].iloc[0]  # Assuming all rows in the group have the same color
        locations = [(row['Latitude'], row['Longitude']) for index, row in group_data.iterrows()]
        folium.PolyLine(locations=locations, color=color_1).add_to(m)
    return m

In [43]:

all_points = read_files()
all_points

{'activity_name': 'cool hike', 'activity_description': 'Lets hike in otaranta its cool', 'activity_date': '2024-03-23', 'activity_schedule': '6.00am', 'activity_time': '00:30', 'activity_location': 'Otaranta 8', 'activity_capacity': 1, 'activity_images': 'images/c8a343f185352af293d19a84e9ea1c09.JPG,images/700c295be4ebc4179e1a82f73dea3dc5.jpg,images/38d309d20762de4d12069cfcfc41b0d3.JPG,images/924a3666427f190f8064c7391b0c3eba.JPG,images/9da098810db066fb29ffbac88cfc321e.HEIC,images/ec04b09c8ce9d0663aa837b3187ea13c.HEIC,', 'activity_link': 'https://www.google.com/maps', 'activity_tweet': 'cool hike in the night', 'activity_age_group': '13-15', 'message4users': 'join me lol', 'activity_gpx': '<?xml version="1.0" encoding="UTF-8"?>\n<gpx xmlns="http://www.topografix.com/GPX/1/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.topografix.com/GPX/1/1 http://www.topografix.com/GPX/1/1/gpx.xsd" version="1.1" creator="gpx.py -- https://github.com/tkrajina/gpxp

[{'Latitude': 60.156789226,
  'Longitude': 24.538536267,
  'Color': '#0d7be5',
  'Number': 0},
 {'Latitude': 60.156690907,
  'Longitude': 24.538660152,
  'Color': '#0d7be5',
  'Number': 0},
 {'Latitude': 60.156564927,
  'Longitude': 24.539016634,
  'Color': '#0d7be5',
  'Number': 0},
 {'Latitude': 60.156426625,
  'Longitude': 24.539389964,
  'Color': '#0d7be5',
  'Number': 0},
 {'Latitude': 60.156322354,
  'Longitude': 24.539679307,
  'Color': '#0d7be5',
  'Number': 0},
 {'Latitude': 60.156207522,
  'Longitude': 24.539973428,
  'Color': '#0d7be5',
  'Number': 0},
 {'Latitude': 60.156033346,
  'Longitude': 24.540373916,
  'Color': '#0d7be5',
  'Number': 0},
 {'Latitude': 60.155974422,
  'Longitude': 24.540516157,
  'Color': '#0d7be5',
  'Number': 0},
 {'Latitude': 60.155801838,
  'Longitude': 24.540863251,
  'Color': '#0d7be5',
  'Number': 0},
 {'Latitude': 60.155726401,
  'Longitude': 24.540905161,
  'Color': '#0d7be5',
  'Number': 0},
 {'Latitude': 60.155458599,
  'Longitude': 24.5410

In [ ]:
plot_map(all_points)